In [9]:
import sys
sys.path.append('python')


import pandas as pd
from glob import glob 

from globals import cities, cities_all


data_source = 'python/tests/unit/fixtures/'
data_dest = 'python/tests/unit/fixtures/'
#current_time = time_now_pandas()
# !! overwrite time during code dev
#current_time = pd.to_datetime('2020-07-08 08:59')
current_time = pd.to_datetime('2020-07-08 20:59')
current_time_str = str(current_time)


In [10]:
import os
import shutil
def clean_up_stat_files(data_dest):
    data_dest_cd = data_dest + "data_cumulative/city_date/"
    for city in cities + cities_all:
        for date in ['2020-07-08']:
            city_date_stats_path = f'{data_dest_cd}{city}/stats/'
            filename = f'{city_date_stats_path}stats_{date}.json'
            try: 
                os.remove(filename)
                print('deleted:', filename) 
            except:
                pass


def copy_initial_logfile(data_dest):
    path0 = f'{data_dest}data_processing_log/log_stats_init.csv'
    path1 = f'{data_dest}data_processing_log/log_stats.csv'
    try:
        shutil.copyfile(path0, path1)
    except:
        pass

In [3]:
#clean_up_stat_files(data_dest)
#copy_initial_logfile(data_dest)

In [11]:
def get_same_day_hours_up_to_timestamp(timestamp):
    timestamp = pd.to_datetime(timestamp).floor('h')
    hours = [str(timestamp + pd.Timedelta(-h, unit='h')) for h in range(0, timestamp.hour+1)]
    return hours[::-1]

stats_datetime_candidates = get_same_day_hours_up_to_timestamp(current_time)
stats_datetime_candidates

['2020-07-08 00:00:00',
 '2020-07-08 01:00:00',
 '2020-07-08 02:00:00',
 '2020-07-08 03:00:00',
 '2020-07-08 04:00:00',
 '2020-07-08 05:00:00',
 '2020-07-08 06:00:00',
 '2020-07-08 07:00:00',
 '2020-07-08 08:00:00',
 '2020-07-08 09:00:00',
 '2020-07-08 10:00:00',
 '2020-07-08 11:00:00',
 '2020-07-08 12:00:00',
 '2020-07-08 13:00:00',
 '2020-07-08 14:00:00',
 '2020-07-08 15:00:00',
 '2020-07-08 16:00:00',
 '2020-07-08 17:00:00',
 '2020-07-08 18:00:00',
 '2020-07-08 19:00:00',
 '2020-07-08 20:00:00']

In [12]:
from utilities import path_ends_with_slash, append_a_list_to_csv_file
from pandas import read_csv

class LogFileUtilityCSV:
    def __init__(self, file_location, filename, 
                 varname='name'):
        self.file_location = path_ends_with_slash(file_location)
        self.filename = filename
        self.path_and_filename = f'{self.file_location}{self.filename}'
        self.varname=varname
    
    def add_candidate_records(self, records):
        self.candidate_records = list(records)
    
    def read_existing_records(self):
        self.existing_records = list(read_csv(self.path_and_filename)[self.varname])
    
    def get_new_records(self):
        if not hasattr(self, 'candidate_records'): raise ValueError('Please add candiate records.')
        self.new_records = [record for record in self.candidate_records if record not in self.existing_records]
    
    def append_new_records(self, alt_filename=None):
        file = self.path_and_filename if alt_filename is None else alt_filename
        append_a_list_to_csv_file(self.new_records, self.varname, file)


logfile_stats = LogFileUtilityCSV(f'{data_dest}data_processing_log/', 'log_stats.csv', varname='datetime')     
logfile_stats.add_candidate_records(stats_datetime_candidates)
logfile_stats.read_existing_records()
logfile_stats.get_new_records()
logfile_stats.new_records
#logfile_stats.append_new_records(f'{data_dest}data_processing_log/log_stats_updated.csv')

['2020-07-08 09:00:00',
 '2020-07-08 10:00:00',
 '2020-07-08 11:00:00',
 '2020-07-08 12:00:00',
 '2020-07-08 13:00:00',
 '2020-07-08 14:00:00',
 '2020-07-08 15:00:00',
 '2020-07-08 16:00:00',
 '2020-07-08 17:00:00',
 '2020-07-08 18:00:00',
 '2020-07-08 19:00:00',
 '2020-07-08 20:00:00']

In [13]:
logfile_stats.existing_records

['2020-07-09 00:00:00',
 '2020-07-09 01:00:00',
 '2020-07-09 02:00:00',
 '2020-07-09 03:00:00',
 '2020-07-08 00:00:00',
 '2020-07-08 01:00:00',
 '2020-07-08 02:00:00',
 '2020-07-08 03:00:00',
 '2020-07-08 04:00:00',
 '2020-07-08 05:00:00',
 '2020-07-08 06:00:00',
 '2020-07-08 07:00:00',
 '2020-07-08 08:00:00']

In [14]:
def get_hour_value_from_timestamps(timestamps):
    return [datetime.hour for datetime in pd.to_datetime(timestamps)]

process_hours = get_hour_value_from_timestamps(logfile_stats.new_records)
process_hours

[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [15]:
from process_city_date_stats import *
data_dest = 'python/tests/unit/fixtures/'
data_dest_cd = data_dest + "data_cumulative/city_date/"
for city in cities[:5] + cities_all[:2]:
#for city in cities + cities_all:
    create_or_update_city_date_stats(city, data_dest_cd, current_time, process_hours)

logfile_stats.append_new_records(f'{data_dest}data_processing_log/log_stats.csv')




--------------- Minneapolis ---------------:

stat_words
         subset                                      token_counter  count
0        today  {'painting': 234, 'hate': 234, 'black': 167, '...    200
1    yesterday  {'protest': 178, 'minneapolis': 104, 'breaking...    700
2   seven_days  {'protest': 5718, 'minneapolis': 3756, 'right'...  10000
3       hour_9  {'minneapolis': 11, 'it’s': 5, 'crazy': 5, 'fi...     12
4      hour_10                                                 {}      0
5      hour_11                                                 {}      0
6      hour_12                                                 {}      0
7      hour_13                                                 {}      0
8      hour_14                                                 {}      0
9      hour_15  {'minneapolis': 21, 'die': 9, 'it’s': 8, 'craz...     24
10     hour_16                                                 {}      0
11     hour_17                                                 {


stat_words
         subset                                      token_counter  count
0        today  {'police': 49, 'raid': 37, 'breonna': 37, 'los...     94
1    yesterday  {'la': 40, 'alyssa': 22, 'black': 11, 'also': ...    200
2   seven_days  {'la': 3437, 'los': 387, 'protest': 132, 'ange...   7200
3       hour_9  {'nadie': 3, 'la': 2, 'comedian': 1, 'stuff': ...      2
4      hour_10                                                 {}      0
5      hour_11                                                 {}      0
6      hour_12                                                 {}      0
7      hour_13                                                 {}      0
8      hour_14                                                 {}      0
9      hour_15  {'la': 6, 'community': 6, '#losangeles': 5, 'b...     12
10     hour_16                                                 {}      0
11     hour_17                                                 {}      0
12     hour_18                        


stat_words
         subset                                      token_counter  count
0        today  {'inter': 45, 'orlando': 43, 'city': 43, 'play...    100
1    yesterday  {'#miami': 5, 'black': 4, 'kneel': 4, 'neck': ...      9
2   seven_days  {'miami': 147, 'woman': 42, '#miami': 99, 'cop...    300
3       hour_9                                                 {}      0
4      hour_10                                                 {}      0
5      hour_11                                                 {}      0
6      hour_12                                                 {}      0
7      hour_13                                                 {}      0
8      hour_14                                                 {}      0
9      hour_15  {'state': 10, 'attorney': 10, 'never': 10, 'ch...     21
10     hour_16                                                 {}      0
11     hour_17                                                 {}      0
12     hour_18                        


stat_words
         subset                                      token_counter  count
0        today  {'michael': 161, 'breonna': 7, 'protest': 97, ...   2000
1    yesterday  {'⁦⁩': 88, 'hate': 121, 'breaking:': 107, '#ma...   4000
2   seven_days  {'right': 514, 'protest': 632, 'nothing': 416,...  10000
3       hour_9  {'michael': 10, 'holding': 10, 'educate': 9, '...     49
4      hour_10                                                 {}      0
5      hour_11                                                 {}      0
6      hour_12                                                 {}      0
7      hour_13                                                 {}      0
8      hour_14                                                 {}      0
9      hour_15  {'thing': 26, 'message,': 25, '#blacklivesmatt...    300
10     hour_16                                                 {}      0
11     hour_17                                                 {}      0
12     hour_18                        

In [ ]:
import os

for city in cities + cities_all:
    for date in ['2020-07-09']:
        city_date_stats_path = f'{data_dest_cd}{city}/stats'
        filename = f'{city_date_stats_path}/stats_{date.replace(" ","_")}.json'
        try: 
            os.remove(filename)
            print('deleted:', filename) 
        except:
            pass



In [ ]:
import  shutil
path0 = f'{data_dest}data_processing_log/log_stats_init.csv'
path1 = f'{data_dest}data_processing_log/log_stats.csv'

try:
    shutil.copyfile(path0, path1)
except:
    pass


In [ ]:
import json
filename = 'python/tests/unit/fixtures/data_cumulative/city_date/Minneapolis/stats/stats_2020-07-09.json'
with open(filename) as f:
    stats = json.load(f) 
    
stat_sentiments, stat_emotions, stat_words, top_tweets, top_users \
     = [pd.read_json(stats[stat], **json_split) 
        for stat in ['stat_sentiments', 'stat_emotions', 'stat_words', 'top_tweets', 'top_users']]
  

In [ ]:
from pandas.testing import assert_frame_equal

assert_frame_equal(stat_sentiments,stat_sentiments)


In [ ]:
stat_sentiments



In [ ]:
stat_words

In [ ]:
top_tweets

In [ ]:
top_users